## Preprocessing

для поиска будем пользоваться word2vec, и чтобы размеры файлов не были такими большими, оставим только слова, которые есть в исходных данных.

Исходные файлы:

* `wiki-news-300d-1M.vec` из https://dl.fbaipublicfiles.com/fasttext/vectors-english/wiki-news-300d-1M.vec.zip

* `english_cleaned_lyrics.csv` из https://github.com/hiteshyalamanchili/SongGenreClassification/tree/master/dataset

In [1]:
import numpy as np
import pandas as pd

lyrics = pd.read_csv(r'english_cleaned_lyrics.csv')
lyrics = lyrics[['song','artist','lyrics']]
lyrics.loc[lyrics.song.isna(), 'song'] = 'green-onions'

Соберем все слова, используемые в песнях  и названиях

In [2]:
def capitalize(x):
    s = x.split('-')
    return ' '.join([w[0].upper() + w[1:] for w in s])

lyrics.artist = lyrics.artist.apply(capitalize)
lyrics.song = lyrics.song.apply(capitalize)

In [3]:
def is_letter_or_space(x):
    return ord('a') <= ord(x) <= ord('z') or x == ' '

def filter_string(s):
    return ''.join(filter(is_letter_or_space, map(str.lower, s)))

filtered_lyrics = lyrics.lyrics.apply(filter_string) 

In [4]:
all_words = []

for w in lyrics.song:
    all_words.extend(w.lower().split())

for w in filtered_lyrics:
    all_words.extend(w.split())
    
all_words = set(all_words)

In [5]:
len(all_words)

193185

In [6]:
lyrics.rename(columns={'song':'title'}, inplace = True)
lyrics.to_csv('data.csv', index = False)

Разделим файл на 5, чтобы хватило оперативной памяти

`$ split -l 200000 wiki-news-300d-1M.vec -d --additional-suffix=.csv f`

И оставим в таблицах только те слова, которые встеритлись в песнях

In [7]:
import codecs

for k in range(5):
    data = [[] for _ in range(301)]
    words = set()
    with codecs.open(f"f0{k}.csv", "r", "utf_8_sig" ) as f:
            for line in f:
                l = line.split()
                if len(l)==301:
                    for i, o in enumerate(l):
                        if i == 0:
                            word = o.lower()
                            if word in words:
                                break
                            else:
                                words.add(word)
                            data[i].append(word)
                        else:
                            data[i].append(o)

    labels = list(map(str, ['word'] + list(range(300))))
    x = pd.DataFrame({labels[i] : data[i] for i in range(301)})
    index = [w.lower() in all_words for w in x.word]

    x[index].to_csv(f't0{k}.csv', header= k==0, index=False)

Теперь соберем все в один файл

`$ cat t00.csv t01.csv t02.csv t03.csv t04.csv > words.csv`

Полученые файлы
* `data.csv` таблица с песнями
* `words.csv` word2vec таблица слов